In [ ]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
from bs4 import BeautifulSoup

URL = "https://www.liquor.com/grasshopper-two-cocktail-recipe-5190061"
def get_recipe(url):
    headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.79 Safari/537.36"}
    page = requests.get(url=url, headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")

    header_section = soup.find(id="heading_1-0")
    recipe_name = header_section.find("h1", class_="heading__title").text

    ingredients_section = soup.find(id="section--ingredients_1-0")
    ingredient_list = ingredients_section.find_all("li", class_="structured-ingredients__list-item")
    ingredients = []
    if (len(ingredient_list) != 0):
        for ingredient in ingredient_list:
            parts = ingredient.find_all("span")
            ingredient_text = ""
            for part in parts:
                ingredient_text += part.text + " "
            ingredients.append(ingredient_text)
    else:
        ingredient_list = ingredients_section.find_all("li", class_="simple-list__item js-checkbox-trigger ingredient text-passage")
        for ingredient in ingredient_list:
            ingredients.append(ingredient.text.strip("\n"))
    instructions_section = soup.find(id="structured-project__steps_1-0")
    instructions_list = instructions_section.find_all("p", class_="comp mntl-sc-block mntl-sc-block-html")
    instructions = []
    for instruction in instructions_list:
        instructions.append(instruction.text.strip("\n"))
    if len(recipe_name) ==0 or len(ingredients) == 0 or len(instructions) == 0:
        print (recipe_name, ingredients, instructions, url)
    return recipe_name, ingredients, instructions
get_recipe(URL)

('Grasshopper 2.0',
 ['3/4 ounce Tempus Fugit white creme de menthe\xa0',
  '3/4 ounce Giffard creme de cacao',
  '1/2 ounce Del Maguey Vida mezcal',
  '1/4 ounce Branca Menta',
  '1 barspoon absinthe',
  '1 1/4 ounces heavy cream',
  '1 drop food coloring',
  'Garnish: grated dark chocolate',
  'Garnish: mint sprig'],
 ['Add all ingredients into a shaker with ice and shake until well-chilled.',
  'Strain into a double rocks glass over crushed ice.',
  'Garnish with grated chocolate and a mint sprig, and serve with a straw.'])

In [ ]:
import pandas as pd
with open('cocktailsfromliquor.com.txt',  mode='r', encoding='utf-8-sig') as f:
    lines = f.read().splitlines()
    urls = list(set(lines))
df = pd.DataFrame(columns=["Name", "Ingredients", "Instructions"])
for url in urls:
    name, ingredients, instructions = get_recipe(url)
    data = {"Name":name, "Ingredients":ingredients, "Instructions":instructions}
    df = df.append(data, ignore_index=True)
print(df)

                                          Name  \
0                           Super Sunday Punch   
1                                   Kew Garden   
2                         Corpse Reviver No. 2   
3                                    Hurricane   
4                              Perfect Martini   
..                                         ...   
300                                    Negroni   
301                                 Grand Slam   
302  Watermelon-Blackberry Margarita Popsicles   
303                                  Churchill   
304                                 Caipirinha   

                                           Ingredients  \
0    [25 whole cloves , 6 cinnamon sticks , 1 teasp...   
1    [8 mint leaves, 3 cucumber slices, 1/2 ounce s...   
2    [Absinthe , 3/4 ounce gin , 3/4 ounce Lillet b...   
3    [2 ounces rum , 2 ounces rum , 1 ounce lime ju...   
4    [1 1/2 ounces Beefeater gin, 1 ounce Dolin dry...   
..                                                 

In [ ]:
df.to_pickle("cocktail_dataset.pkl")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import evaluate
predictions = ["hello there general kenobi", "foo bar foobar"]
references = [["hello there general kenobi", "hello there !"], ["foo bar foobar"]]
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)